# BMDS for ToxRefDB - Part 2

The BMDS python interface was developed by Andy Shapiro.

bmds package and information can be found at :
[bmds package GitHub](https://github.com/shapiromatron/bmds)



In [1]:
from copy import deepcopy
from collections import OrderedDict
from datetime import datetime
import itertools
import json
import os
import time
import pandas as pd
import bmds

#os.chdir("---")

In [2]:
# start a timer
start_time = datetime.now()

# Import Datasets and conversting to useable format

In [3]:
# import data filtered from part1
import pandas as pd
cancer = pd.read_csv('cancer_tr.csv', sep=',')
dichotomous = pd.read_csv('dichotomous_tr.csv', sep=',')
continuous = pd.read_csv('continuous_NotBW.csv', sep=',')
weight = pd.read_csv('continuous_BW.csv')

In [4]:
# Getting data into useable format and creating datasets from each row

def dichotomous_dictify(d):
    try:
        return dict(
              id=d.id,
              doses=list(map(float, d.doses.split(','))),
              ns=list(map(int, d.ns.split(','))),
              incidences=list(map(int, d.incidences.split(','))),
        )
    except:
        print('Row {} not included'.format(d.id))
        return None
    

def continuous_dictify(d):
    try:
        return dict(
              id=d.id,
              doses=list(map(float, d.doses.split(','))),
              ns=list(map(int, d.ns.split(','))),
              means=list(map(float, d.means.split(','))),
              stdevs=list(map(float, d.stdevs.split(','))),
        )
    except:
        print('Row {} not included'.format(d.id))
        return None

In [ ]:
dichotomous_datasets = [
    d for d in dichotomous.apply(dichotomous_dictify, axis=1) 
    if d is not None
]

dichotomous_cancer_datasets = [
    d for d in cancer.apply(dichotomous_dictify, axis=1)
    if d is not None
]

continuous_datasets = [
    d for d in continuous.apply(continuous_dictify, axis=1) 
    if d is not None
]

continuous_weight_datasets = [
    d for d in weight.apply(continuous_dictify, axis=1) 
    if d is not None
]

In [6]:
def get_bmds_dataset(dtype, dataset):
    if dtype == bmds.constants.CONTINUOUS:
        cls = bmds.ContinuousDataset
    elif dtype in bmds.constants.DICHOTOMOUS_DTYPES:
        cls = bmds.DichotomousDataset
    return cls(**dataset)

 # Running Dataset Through BMDS

### Dichotomous Dataset

### Dichotomous-Cancer Dataset

In [14]:
# To run for 5% BMR
def execute_with_dose_drops_dichotomous_cancer(dtype, original_dataset):
    dataset = deepcopy(original_dataset)
    bmds_dataset = get_bmds_dataset(dtype, dataset)
    session = bmds.BMDS.latest_version(dtype, dataset=bmds_dataset)
    bmr_types = bmds.constants.BMR_CROSSWALK[bmds.constants.DICHOTOMOUS_CANCER]
    overrides = { 
      'bmr': 0.05,
      'bmr_type': bmr_types['Extra']
      }
    session.add_default_models(global_overrides=overrides)
    session.execute_and_recommend(drop_doses=True)
    print(dataset)
    return session

dichotomous_cancer5bmr = [
    execute_with_dose_drops_dichotomous_cancer
    (bmds.constants.DICHOTOMOUS_CANCER, dataset) 
    for dataset in dichotomous_cancer_datasets
]

dicts = [
    session.to_dict(1)
    for session in dichotomous_cancer5bmr
]

with open('dichotomous_cancer5bmr.json', 'w') as f:
    json.dump(dicts, f)
    
def get_od():
    # return an ordered defaultdict list
    keys = [
        'dataset_id', 'doses_dropped', 
        
        'model_name', 'model_version', 'has_output',

        'BMD', 'BMDL', 'BMDU', 'CSF',
        'AIC', 'pvalue1', 'pvalue2', 'pvalue3', 'pvalue4',
        'Chi2', 'df', 'residual_of_interest',
        'warnings',

        'logic_bin', 'logic_cautions', 'logic_warnings', 'logic_failures',
        'recommended', 'recommended_variable',

        'dfile', 'outfile',
    ]

    return OrderedDict([(key, list()) for key in keys])

flat = get_od()

for session in itertools.chain(dichotomous_cancer5bmr):

    for model in session.models:
        # dataset-level
        flat['dataset_id'].append(session.dataset.kwargs['id'])
        flat['doses_dropped'].append(session.doses_dropped)
        
        # model-level
        flat['model_name'].append(model.name)
        flat['model_version'].append(model.version)
        flat['has_output'].append(model.has_successfully_executed)
        
        # outputs
        op = getattr(model, 'output', {})
        if op is None:
            op =  {}
        flat['BMD'].append(op.get('BMD'))
        flat['BMDL'].append(op.get('BMDL'))
        flat['BMDU'].append(op.get('BMDU'))
        flat['CSF'].append(op.get('CSF'))
        flat['AIC'].append(op.get('AIC'))
        flat['pvalue1'].append(op.get('p_value1'))
        flat['pvalue2'].append(op.get('p_value2'))
        flat['pvalue3'].append(op.get('p_value3'))
        flat['pvalue4'].append(op.get('p_value4'))
        flat['Chi2'].append(op.get('Chi2'))
        flat['df'].append(op.get('df'))
        flat['residual_of_interest'].append(op.get('residual_of_interest'))
        flat['warnings'].append('\n'.join(op.get('warnings', [])))
        
        # logic
        flat['logic_bin'].append(model.logic_bin)
        flat['logic_cautions'].append('\n'.join(model.logic_notes[0]))
        flat['logic_warnings'].append('\n'.join(model.logic_notes[1]))
        flat['logic_failures'].append('\n'.join(model.logic_notes[2]))
        flat['recommended'].append(model.recommended)
        flat['recommended_variable'].append(model.recommended_variable)
        
        flat['dfile'].append(model.as_dfile())
        flat['outfile'].append(getattr(model, 'outfile', ''))

output_df = pd.DataFrame(flat)
output_df.sort_values(['dataset_id', 'model_name'], inplace=True)

# output filename

OUTPUT_FN = 'ToxRefDB_bmd_dichotomous_cancer5bmr.xlsx' 

fn = os.path.expanduser(OUTPUT_FN)
output_df.to_excel(fn, index=False)


In [ ]:
end_time = datetime.now()
delta = end_time - start_time
print('Total time: {0:2.5} minutes'.format(str(delta.total_seconds()/60.), 2))

In [8]:
# To Run for 10% BMR
def execute_with_dose_drops_dichotomous_cancer(dtype, original_dataset):
    dataset = deepcopy(original_dataset)
    bmds_dataset = get_bmds_dataset(dtype, dataset)
    session = bmds.BMDS.latest_version(dtype, dataset=bmds_dataset)
    bmr_types = bmds.constants.BMR_CROSSWALK[bmds.constants.DICHOTOMOUS_CANCER]
    overrides = { 
      'bmr': 0.1,
      'bmr_type': bmr_types['Extra']
      }
    session.add_default_models(global_overrides=overrides)
    session.execute_and_recommend(drop_doses=True)
    print(dataset)
    return session

dichotomous_cancer10bmr = [
    execute_with_dose_drops_dichotomous_cancer
    (bmds.constants.DICHOTOMOUS_CANCER, dataset) 
    for dataset in dichotomous_cancer_datasets
]

dicts = [
    session.to_dict(1)
    for session in dichotomous_cancer10bmr
]

with open('dichotomous_cancer10bmr.json', 'w') as f:
    json.dump(dicts, f)
    
#output file
def get_od():
    # return an ordered defaultdict list
    keys = [
        'dataset_id', 'doses_dropped', 
        
        'model_name', 'model_version', 'has_output',

        'BMD', 'BMDL', 'BMDU', 'CSF',
        'AIC', 'pvalue1', 'pvalue2', 'pvalue3', 'pvalue4',
        'Chi2', 'df', 'residual_of_interest',
        'warnings',

        'logic_bin', 'logic_cautions', 'logic_warnings', 'logic_failures',
        'recommended', 'recommended_variable',

        'dfile', 'outfile',
    ]

    return OrderedDict([(key, list()) for key in keys])

flat = get_od()

for session in itertools.chain(dichotomous_cancer10bmr):

    for model in session.models:
        # dataset-level
        flat['dataset_id'].append(session.dataset.kwargs['id'])
        flat['doses_dropped'].append(session.doses_dropped)
        
        # model-level
        flat['model_name'].append(model.name)
        flat['model_version'].append(model.version)
        flat['has_output'].append(model.has_successfully_executed)
        
        # outputs
        op = getattr(model, 'output', {})
        if op is None:
            op =  {}
        flat['BMD'].append(op.get('BMD'))
        flat['BMDL'].append(op.get('BMDL'))
        flat['BMDU'].append(op.get('BMDU'))
        flat['CSF'].append(op.get('CSF'))
        flat['AIC'].append(op.get('AIC'))
        flat['pvalue1'].append(op.get('p_value1'))
        flat['pvalue2'].append(op.get('p_value2'))
        flat['pvalue3'].append(op.get('p_value3'))
        flat['pvalue4'].append(op.get('p_value4'))
        flat['Chi2'].append(op.get('Chi2'))
        flat['df'].append(op.get('df'))
        flat['residual_of_interest'].append(op.get('residual_of_interest'))
        flat['warnings'].append('\n'.join(op.get('warnings', [])))
        
        # logic
        flat['logic_bin'].append(model.logic_bin)
        flat['logic_cautions'].append('\n'.join(model.logic_notes[0]))
        flat['logic_warnings'].append('\n'.join(model.logic_notes[1]))
        flat['logic_failures'].append('\n'.join(model.logic_notes[2]))
        flat['recommended'].append(model.recommended)
        flat['recommended_variable'].append(model.recommended_variable)
        
        flat['dfile'].append(model.as_dfile())
        flat['outfile'].append(getattr(model, 'outfile', ''))

output_df = pd.DataFrame(flat)
output_df.sort_values(['dataset_id', 'model_name'], inplace=True)

# output filename

OUTPUT_FN = 'ToxRefDB_bmd_dichotomous_cancer10bmr.xlsx' 

fn = os.path.expanduser(OUTPUT_FN)
output_df.to_excel(fn, index=False)


In [ ]:
end_time = datetime.now()
delta = end_time - start_time
print('Total time: {0:2.5} minutes'.format(str(delta.total_seconds()/60.), 2))

For Dichotomous non-Cancer Endpoints

In [ ]:
# To Run for 10% BMR
def execute_with_dose_drops_dichotomous(dtype, original_dataset):
    dataset = deepcopy(original_dataset)
    bmds_dataset = get_bmds_dataset(dtype, dataset)
    session = bmds.BMDS.latest_version(dtype, dataset=bmds_dataset)
    bmr_types = bmds.constants.BMR_CROSSWALK[bmds.constants.DICHOTOMOUS]
    overrides = { 
      'bmr': 0.1,
      'bmr_type': bmr_types['Extra']
      }
    session.add_default_models(global_overrides=overrides)
    session.execute_and_recommend(drop_doses=True)
    print(dataset)
    return session

dichotomous_10bmr = [
    execute_with_dose_drops_dichotomous(bmds.constants.DICHOTOMOUS, dataset) 
    for dataset in dichotomous_datasets
]

dicts = [
    session.to_dict(1)
    for session in dichotomous_10bmr
]

with open('dichotomous_10bmr.json', 'w') as f: 
    json.dump(dicts, f)
    
# Output files

def get_od():
    # return an ordered defaultdict list
    keys = [
        'dataset_id', 'doses_dropped', 
        
        'model_name', 'model_version', 'has_output',

        'BMD', 'BMDL', 'BMDU', 'CSF',
        'AIC', 'pvalue1', 'pvalue2', 'pvalue3', 'pvalue4',
        'Chi2', 'df', 'residual_of_interest',
        'warnings',

        'logic_bin', 'logic_cautions', 'logic_warnings', 'logic_failures',
        'recommended', 'recommended_variable',

        'dfile', 'outfile',
    ]

    return OrderedDict([(key, list()) for key in keys])

flat = get_od()

for session in itertools.chain(dichotomous_10bmr):

    for model in session.models:
        # dataset-level
        flat['dataset_id'].append(session.dataset.kwargs['id'])
        flat['doses_dropped'].append(session.doses_dropped)
        
        # model-level
        flat['model_name'].append(model.name)
        flat['model_version'].append(model.version)
        flat['has_output'].append(model.has_successfully_executed)
        
        # outputs
        op = getattr(model, 'output', {})
        if op is None:
            op =  {}
        flat['BMD'].append(op.get('BMD'))
        flat['BMDL'].append(op.get('BMDL'))
        flat['BMDU'].append(op.get('BMDU'))
        flat['CSF'].append(op.get('CSF'))
        flat['AIC'].append(op.get('AIC'))
        flat['pvalue1'].append(op.get('p_value1'))
        flat['pvalue2'].append(op.get('p_value2'))
        flat['pvalue3'].append(op.get('p_value3'))
        flat['pvalue4'].append(op.get('p_value4'))
        flat['Chi2'].append(op.get('Chi2'))
        flat['df'].append(op.get('df'))
        flat['residual_of_interest'].append(op.get('residual_of_interest'))
        flat['warnings'].append('\n'.join(op.get('warnings', [])))
        
        # logic
        flat['logic_bin'].append(model.logic_bin)
        flat['logic_cautions'].append('\n'.join(model.logic_notes[0]))
        flat['logic_warnings'].append('\n'.join(model.logic_notes[1]))
        flat['logic_failures'].append('\n'.join(model.logic_notes[2]))
        flat['recommended'].append(model.recommended)
        flat['recommended_variable'].append(model.recommended_variable)
        
        flat['dfile'].append(model.as_dfile())
        flat['outfile'].append(getattr(model, 'outfile', ''))

output_df = pd.DataFrame(flat)
output_df.sort_values(['dataset_id', 'model_name'], inplace=True)

# output filename

OUTPUT_FN = 'ToxRefDB_bmd_dichotomous_10bmr.xlsx' 

fn = os.path.expanduser(OUTPUT_FN)
output_df.to_excel(fn, index=False)

In [ ]:
end_time = datetime.now()
delta = end_time - start_time
print('Total time: {0:2.5} minutes'.format(str(delta.total_seconds()/60.), 2))

In [ ]:
# To Run for 5% BMR

def execute_with_dose_drops_dichotomous(dtype, original_dataset):
    dataset = deepcopy(original_dataset)
    bmds_dataset = get_bmds_dataset(dtype, dataset)
    session = bmds.BMDS.latest_version(dtype, dataset=bmds_dataset)
    bmr_types = bmds.constants.BMR_CROSSWALK[bmds.constants.DICHOTOMOUS]
    overrides = { 
      'bmr': 0.05,
      'bmr_type': bmr_types['Extra']
      }
    session.add_default_models(global_overrides=overrides)
    session.execute_and_recommend(drop_doses=True)
    print(dataset)
    return session

dichotomous_05bmr = [
    execute_with_dose_drops_dichotomous(bmds.constants.DICHOTOMOUS, dataset) 
    for dataset in dichotomous_datasets
]

dicts = [
    session.to_dict(1)
    for session in dichotomous_05bmr
]

with open('dichotomous_05bmr.json', 'w') as f: 
    json.dump(dicts, f)
    
# output file
def get_od():
    # return an ordered defaultdict list
    keys = [
        'dataset_id', 'doses_dropped', 
        
        'model_name', 'model_version', 'has_output',

        'BMD', 'BMDL', 'BMDU', 'CSF',
        'AIC', 'pvalue1', 'pvalue2', 'pvalue3', 'pvalue4',
        'Chi2', 'df', 'residual_of_interest',
        'warnings',

        'logic_bin', 'logic_cautions', 'logic_warnings', 'logic_failures',
        'recommended', 'recommended_variable',

        'dfile', 'outfile',
    ]

    return OrderedDict([(key, list()) for key in keys])

flat = get_od()

for session in itertools.chain(dichotomous_05bmr):

    for model in session.models:
        # dataset-level
        flat['dataset_id'].append(session.dataset.kwargs['id'])
        flat['doses_dropped'].append(session.doses_dropped)
        
        # model-level
        flat['model_name'].append(model.name)
        flat['model_version'].append(model.version)
        flat['has_output'].append(model.has_successfully_executed)
        
        # outputs
        op = getattr(model, 'output', {})
        if op is None:
            op =  {}
        flat['BMD'].append(op.get('BMD'))
        flat['BMDL'].append(op.get('BMDL'))
        flat['BMDU'].append(op.get('BMDU'))
        flat['CSF'].append(op.get('CSF'))
        flat['AIC'].append(op.get('AIC'))
        flat['pvalue1'].append(op.get('p_value1'))
        flat['pvalue2'].append(op.get('p_value2'))
        flat['pvalue3'].append(op.get('p_value3'))
        flat['pvalue4'].append(op.get('p_value4'))
        flat['Chi2'].append(op.get('Chi2'))
        flat['df'].append(op.get('df'))
        flat['residual_of_interest'].append(op.get('residual_of_interest'))
        flat['warnings'].append('\n'.join(op.get('warnings', [])))
        
        # logic
        flat['logic_bin'].append(model.logic_bin)
        flat['logic_cautions'].append('\n'.join(model.logic_notes[0]))
        flat['logic_warnings'].append('\n'.join(model.logic_notes[1]))
        flat['logic_failures'].append('\n'.join(model.logic_notes[2]))
        flat['recommended'].append(model.recommended)
        flat['recommended_variable'].append(model.recommended_variable)
        
        flat['dfile'].append(model.as_dfile())
        flat['outfile'].append(getattr(model, 'outfile', ''))

output_df = pd.DataFrame(flat)
output_df.sort_values(['dataset_id', 'model_name'], inplace=True)

# output filename

OUTPUT_FN = 'ToxRefDB_bmd_dichotomous_05bmr.xlsx' 

fn = os.path.expanduser(OUTPUT_FN)
output_df.to_excel(fn, index=False)

In [24]:
end_time = datetime.now()
delta = end_time - start_time
print('Total time: {0:2.5} minutes'.format(str(delta.total_seconds()/60.), 2))

Total time: 15.09 minutes


### Continuous Dataset
#### Continuous Non-Body/Organ weight

In [ ]:
# The BMR used here is 1SD

def execute_with_dose_drops_continuous(dtype, original_dataset):
    dataset = deepcopy(original_dataset)
    bmds_dataset = get_bmds_dataset(dtype, dataset)
    session = bmds.BMDS.latest_version(dtype, dataset=bmds_dataset)
    bmr_types = bmds.constants.BMR_CROSSWALK[bmds.constants.CONTINUOUS]
    overrides = { 
      'bmr': 1,
      'bmr_type': bmr_types['Std. Dev.']
       }
    session.add_default_models(global_overrides=overrides)
    session.execute_and_recommend(drop_doses=True)
    print(dataset)
    return session

continuous_1sd = [
    execute_with_dose_drops_continuous(bmds.constants.CONTINUOUS, dataset) 
    for dataset in continuous_datasets
]

dicts = [
    session.to_dict(1)
    for session in continuous_1sd
]
with open('continuous_1sd.json', 'w') as f: 
    json.dump(dicts, f)
    
    
##Output files  
def get_od():
    # return an ordered defaultdict list
    keys = [
        'dataset_id', 'doses_dropped', 
        
        'model_name', 'model_version', 'has_output',

        'BMD', 'BMDL', 'BMDU', 'CSF',
        'AIC', 'pvalue1', 'pvalue2', 'pvalue3', 'pvalue4',
        'Chi2', 'df', 'residual_of_interest',
        'warnings',

        'logic_bin', 'logic_cautions', 'logic_warnings', 'logic_failures',
        'recommended', 'recommended_variable',

        'dfile', 'outfile',
    ]

    return OrderedDict([(key, list()) for key in keys])

flat = get_od()

for session in itertools.chain(continuous_1sd):

    for model in session.models:
        # dataset-level
        flat['dataset_id'].append(session.dataset.kwargs['id'])
        flat['doses_dropped'].append(session.doses_dropped)
        
        # model-level
        flat['model_name'].append(model.name)
        flat['model_version'].append(model.version)
        flat['has_output'].append(model.has_successfully_executed)
        
        # outputs
        op = getattr(model, 'output', {})
        if op is None:
            op =  {}
        flat['BMD'].append(op.get('BMD'))
        flat['BMDL'].append(op.get('BMDL'))
        flat['BMDU'].append(op.get('BMDU'))
        flat['CSF'].append(op.get('CSF'))
        flat['AIC'].append(op.get('AIC'))
        flat['pvalue1'].append(op.get('p_value1'))
        flat['pvalue2'].append(op.get('p_value2'))
        flat['pvalue3'].append(op.get('p_value3'))
        flat['pvalue4'].append(op.get('p_value4'))
        flat['Chi2'].append(op.get('Chi2'))
        flat['df'].append(op.get('df'))
        flat['residual_of_interest'].append(op.get('residual_of_interest'))
        flat['warnings'].append('\n'.join(op.get('warnings', [])))
        
        # logic
        flat['logic_bin'].append(model.logic_bin)
        flat['logic_cautions'].append('\n'.join(model.logic_notes[0]))
        flat['logic_warnings'].append('\n'.join(model.logic_notes[1]))
        flat['logic_failures'].append('\n'.join(model.logic_notes[2]))
        flat['recommended'].append(model.recommended)
        flat['recommended_variable'].append(model.recommended_variable)
        
        flat['dfile'].append(model.as_dfile())
        flat['outfile'].append(getattr(model, 'outfile', ''))

output_df = pd.DataFrame(flat)
output_df.sort_values(['dataset_id', 'model_name'], inplace=True)

# output filename
OUTPUT_FN = 'ToxRefDB_bmd_continuous_1sd.xlsx' 

fn = os.path.expanduser(OUTPUT_FN)
output_df.to_excel(fn, index=False)

In [ ]:
end_time = datetime.now()
delta = end_time - start_time
print('Total time: {0:2.5} minutes'.format(str(delta.total_seconds()/60.), 2))

#### Continuous Body/Organ weight

In [ ]:
# The BMR used here is 10% Relative Deviation

def execute_with_dose_drops_continuousBW(dtype, original_dataset):
    dataset = deepcopy(original_dataset)
    bmds_dataset = get_bmds_dataset(dtype, dataset)
    session = bmds.BMDS.latest_version(dtype, dataset=bmds_dataset)
    bmr_types = bmds.constants.BMR_CROSSWALK[bmds.constants.CONTINUOUS]
    overrides = { 
      'bmr': 0.1,
      'bmr_type': bmr_types['Rel. Dev.']
       }
    session.add_default_models(global_overrides=overrides)
    session.execute_and_recommend(drop_doses=True)
    print(dataset)
    return session

continuousBW_10rd = [
    execute_with_dose_drops_continuousBW(bmds.constants.CONTINUOUS, dataset) 
    for dataset in continuous_weight_datasets 
]

dicts = [
    session.to_dict(1)
    for session in continuousBW_10rd
]
with open('continuousBW_10rd.json', 'w') as f: 
    json.dump(dicts, f)

# output files

def get_od():
    # return an ordered defaultdict list
    keys = [
        'dataset_id', 'doses_dropped', 
        
        'model_name', 'model_version', 'has_output',

        'BMD', 'BMDL', 'BMDU', 'CSF',
        'AIC', 'pvalue1', 'pvalue2', 'pvalue3', 'pvalue4',
        'Chi2', 'df', 'residual_of_interest',
        'warnings',

        'logic_bin', 'logic_cautions', 'logic_warnings', 'logic_failures',
        'recommended', 'recommended_variable',

        'dfile', 'outfile',
    ]

    return OrderedDict([(key, list()) for key in keys])

flat = get_od()

for session in itertools.chain(continuousBW_10rd):


    for model in session.models:
        # dataset-level
        flat['dataset_id'].append(session.dataset.kwargs['id'])
        flat['doses_dropped'].append(session.doses_dropped)
        
        # model-level
        flat['model_name'].append(model.name)
        flat['model_version'].append(model.version)
        flat['has_output'].append(model.has_successfully_executed)
        
        # outputs
        op = getattr(model, 'output', {})
        if op is None:
            op =  {}
        flat['BMD'].append(op.get('BMD'))
        flat['BMDL'].append(op.get('BMDL'))
        flat['BMDU'].append(op.get('BMDU'))
        flat['CSF'].append(op.get('CSF'))
        flat['AIC'].append(op.get('AIC'))
        flat['pvalue1'].append(op.get('p_value1'))
        flat['pvalue2'].append(op.get('p_value2'))
        flat['pvalue3'].append(op.get('p_value3'))
        flat['pvalue4'].append(op.get('p_value4'))
        flat['Chi2'].append(op.get('Chi2'))
        flat['df'].append(op.get('df'))
        flat['residual_of_interest'].append(op.get('residual_of_interest'))
        flat['warnings'].append('\n'.join(op.get('warnings', [])))
        
        # logic
        flat['logic_bin'].append(model.logic_bin)
        flat['logic_cautions'].append('\n'.join(model.logic_notes[0]))
        flat['logic_warnings'].append('\n'.join(model.logic_notes[1]))
        flat['logic_failures'].append('\n'.join(model.logic_notes[2]))
        flat['recommended'].append(model.recommended)
        flat['recommended_variable'].append(model.recommended_variable)
        
        flat['dfile'].append(model.as_dfile())
        flat['outfile'].append(getattr(model, 'outfile', ''))

output_df = pd.DataFrame(flat)
output_df.sort_values(['dataset_id', 'model_name'], inplace=True)

# output filename

OUTPUT_FN = 'ToxRefDB_bmd_continuousBW_10rd.xlsx' 

fn = os.path.expanduser(OUTPUT_FN)
output_df.to_excel(fn, index=False)
    

In [ ]:
end_time = datetime.now()
delta = end_time - start_time
print('Total time: {0:2.5} minutes'.format(str(delta.total_seconds()/60.), 2))